<a href="https://colab.research.google.com/github/jorisschellekens/borb-google-colab-examples/blob/main/using_borb_to_create_a_word_search_puzzle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ![borb logo](https://github.com/jorisschellekens/borb/raw/master/logo/borb_64.png) Using `borb` to create a word search puzzle in PDF

[`borb`](https://github.com/jorisschellekens/borb) is a library for reading, creating and manipulating PDF files in python. borb was created in 2020 by Joris Schellekens and is still in active development. Check out the [GitHub repository](https://github.com/jorisschellekens/borb), or the [borb website](https://borbpdf.com).

Let's start by installing `borb`

In [1]:
pip install borb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.3 MB 4.8 MB/s 
     |████████████████████████████████| 43 kB 1.0 MB/s 
     |████████████████████████████████| 62 kB 803 kB/s 
     |████████████████████████████████| 930 kB 37.2 MB/s 
     |████████████████████████████████| 212 kB 39.6 MB/s 
  Created wheel for qrcode: filename=qrcode-7.3.1-py3-none-any.whl size=40402 sha256=dfba0fe2b519da3bbedb93a8417bddd7fd6ab1dd60c5b3fac194487f2f712f5f
  Stored in directory: /root/.cache/pip/wheels/93/d7/39/a4111be2cfb8e679938aa671a37888b6afb1f9e7d748e94492
Successfully built qrcode
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency confli

We need to build a model to represent our puzzle. I'm going to briefly do that here, and implement a rather naïve search algorithm to populate our word search.

In [2]:
import copy
import random
import typing


class WordPuzzleGrid:

    def __init__(self, words: typing.List[str],
                 width: typing.Optional[int] = None,
                 height: typing.Optional[int] = None):
        self._words: typing.List[str] = [w.upper() for w in words]
        self._solved_grid: typing.Optional[typing.List[typing.List[str]]] = None

        # permissions
        self._allow_horizontal: bool = True
        self._allow_vertical: bool = True
        self._allow_horizontal_reverse: bool = True
        self._allow_vertical_reverse: bool = True

        # init empty grid
        n = max([len(w) for w in words])
        self._width = width or int(n * 1.5)
        self._height = height or int(n * 1.5)
        self.fill_word_search_grid(self._words, [['.' for _ in range(0, self._width)] for _ in range(0, self._height)])

        # fill remainder of grid with random letters
        self._solved_grid = [[c if c != '.' else random.choice('ABCDEFGHIJKLMNOPQRSTUVWXYZ') for c in r] for r in self._solved_grid]

    def fill_word_search_grid(self, words_todo: typing.List[str], words_done: typing.List[typing.List[str]]):

        # a solution has been found
        if len(words_todo) == 0:
            self._solved_grid = copy.deepcopy(words_done)
            return

        # a solution has already been found
        if self._solved_grid is not None:
            return

        # attempt to place 1 more word
        w = words_todo.pop()

        # attempt to place horizontally
        if self._allow_horizontal:
            ts: typing.List[typing.Tuple[int, int]] = []
            for row in range(0, len(words_done)):
                for col in range(0, len(words_done[row]) - len(w) + 1):
                    ts.append((row, col))
            random.shuffle(ts)
            for (row, col) in ts:
                can_match = all([words_done[row][col + i] == '.' or words_done[row][col + i] == w[i] for i in range(0, len(w))])
                if not can_match:
                    continue
                before_placement = words_done[row][col:col+len(w)]
                words_done[row][col:col+len(w)] = [c for c in w]
                self.fill_word_search_grid(words_todo, words_done)
                words_done[row][col:col+len(w)] = [c for c in before_placement]

        # attempt to place vertically
        if self._allow_vertical:
            ts: typing.List[typing.Tuple[int, int]] = []
            for col in range(0, len(words_done[0])):
                for row in range(0, len(words_done) - len(w) + 1):
                    ts.append((row, col))
            random.shuffle(ts)
            for (row, col) in ts:
                can_match = all([words_done[row+i][col] == '.' or words_done[row+i][col] == w[i] for i in range(0, len(w))])
                if not can_match:
                    continue
                before_placement = [words_done[row+i][col] for i in range(0, len(w))]
                for i in range(0, len(w)):
                    words_done[row+i][col] = w[i]
                self.fill_word_search_grid(words_todo, words_done)
                for i in range(0, len(w)):
                    words_done[row+i][col] = before_placement[i]

        # attempt to place horizontally (reversed)
        if self._allow_horizontal_reverse:
            w2 = w[::-1]
            ts: typing.List[typing.Tuple[int, int]] = []
            for row in range(0, len(words_done)):
                for col in range(0, len(words_done[row]) - len(w) + 1):
                    ts.append((row, col))
            random.shuffle(ts)
            for (row, col) in ts:
                can_match = all([words_done[row][col + i] == '.' or words_done[row][col + i] == w2[i] for i in range(0, len(w))])
                if not can_match:
                    continue
                before_placement = words_done[row][col:col+len(w)]
                words_done[row][col:col+len(w)] = [c for c in w2]
                self.fill_word_search_grid(words_todo, words_done)
                words_done[row][col:col+len(w)] = [c for c in before_placement]

        # attempt to place vertically (reversed)
        if self._allow_vertical_reverse:
            w2 = w[::-1]
            ts: typing.List[typing.Tuple[int, int]] = []
            for col in range(0, len(words_done[0])):
                for row in range(0, len(words_done) - len(w) + 1):
                    ts.append((row, col))
            random.shuffle(ts)
            for (row, col) in ts:
                can_match = all([words_done[row+i][col] == '.' or words_done[row+i][col] == w2[i] for i in range(0, len(w))])
                if not can_match:
                    continue
                before_placement = [words_done[row+i][col] for i in range(0, len(w))]
                for i in range(0, len(w)):
                    words_done[row+i][col] = w2[i]
                self.fill_word_search_grid(words_todo, words_done)
                for i in range(0, len(w)):
                    words_done[row+i][col] = before_placement[i]

        # restore
        words_todo.append(w)

    def at(self, row: int, col: int) -> str:
      return self._solved_grid[row][col]

    def __str__(self):
        return "".join(["".join([c for c in row]) + "\n" for row in self._solved_grid])


We are going to be using a custom font for the title of this PDF. So let's start by downloading it;

In [3]:
from borb.pdf.canvas.font.simple_font.true_type_font import TrueTypeFont  
from borb.pdf.canvas.font.font import Font

# Download Font
import requests
with open('IndieFlower-Regular.ttf', 'wb') as ffh:
  ffh.write(requests.get("https://github.com/google/fonts/blob/main/ofl/indieflower/IndieFlower-Regular.ttf?raw=true", allow_redirects=True).content)

Now we can build the `Document` using our custom `Font` and the `WordPuzzleGrid` class we defined earlier.

In [4]:
from borb.pdf.document.document import Document
from borb.pdf.page.page import Page
from borb.pdf.pdf import PDF
from borb.pdf.canvas.layout.page_layout.multi_column_layout import SingleColumnLayout
from borb.pdf.canvas.layout.page_layout.page_layout import PageLayout
from borb.pdf.canvas.layout.text.paragraph import Paragraph
from borb.pdf.canvas.layout.image.image import Image
from borb.pdf.canvas.layout.table.fixed_column_width_table import FixedColumnWidthTable

# Font
from borb.pdf.canvas.color.color import HexColor

# Python
from decimal import Decimal
from pathlib import Path  


# create Document
doc: Document = Document()

page: Page = Page()
doc.add_page(page)

# set PageLayout
layout: PageLayout = SingleColumnLayout(page)

# add title
layout.add(Paragraph('Word Search', 
                     font_color=HexColor('#19647E'),
                      font=TrueTypeFont.true_type_font_from_file(Path("IndieFlower-Regular.ttf")),
                      font_size=Decimal(20)))
  
# add explanation
layout.add(Paragraph('Can you find all the animals hidden in this word search? Words can be horizontal, diagonal, left-to-right and down-to-up.',
                     font_color=HexColor('#28AFB0')))

# build puzzle
animals = ["aardvark", "baboon","cat","dog", "elephant",
          "flamingo", "gnu", "horse", "jackal", "llama",
          "gnat", "rat", "tiger", "monkey", "rhino",
          "hippo", "mouse", "seahorse", "chimpansee"]
word_puzzle_grid: WordPuzzleGrid = WordPuzzleGrid(animals, 13, 13)

# build Table using cells
table:FixedColumnWidthTable = FixedColumnWidthTable(number_of_rows=13, number_of_columns=13)
for i in range(0, 13):
  for j in range(0, 13):
    table.add(Paragraph(word_puzzle_grid.at(i,j)))

# set some global (all cell) table properties
table.set_padding_on_all_cells(Decimal(5), Decimal(5), Decimal(5), Decimal(5))
table.no_borders()

# add to PageLayout
layout.add(table)

# add list of animals to cross out
layout.add(Paragraph("".join([a + ", " for a in animals])[:-2] ))

# write Document
with open("output.pdf", "wb") as pdf_file_handle:
  PDF.dumps(pdf_file_handle, doc)